### A Simple RAG Implementation

In [3]:
# Install necessary dependencies
!pip install unstructured libmagic python-magic python-magic-bin langchain langchain-community

In [4]:
# Initialize llama2 model using Ollama
from langchain_community.llms import Ollama
MODEL="llama3.3"
llama_llm = Ollama(model=MODEL)

In [5]:
# Necessary Imports
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Step 1: Load PDF files using PyPDFLoader
def load_pdfs(folder_path):
    documents = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(folder_path, file_name))
            documents.extend(loader.load())
    return documents

# Step 2: Chunk documents for better retrieval performance
def chunk_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(documents)
    return chunks

# Step 3: Embed the chunks using BERT-based Hugging Face embeddings
def embed_chunks_with_bert(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    chunk_texts = [chunk.page_content for chunk in chunks]
    vectors = embeddings.embed_documents(chunk_texts)
    return vectors, chunk_texts

# Step 4: Index embeddings in a FAISS vector store
def index_embeddings(texts):
    vector_store = FAISS.from_texts(texts, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
    return vector_store

# Step 5: Set up the LLaMA model
def load_llama_model(model_name="llama2"):
    llama_llm = Ollama(model=model_name)
    return llama_llm
    

# Step 6: Create a RAG system
def create_rag_system(vector_store, llama_llm):
    retriever = vector_store.as_retriever()
    rag_chain = RetrievalQA.from_chain_type(llm=llama_llm, retriever=retriever)
    return rag_chain

# Main function to build and run the RAG pipeline


folder_path = "Documentation"  

print("Loading PDF files...")
documents = load_pdfs(folder_path)

# Chunk documents
print("Splitting documents into chunks...")
chunks = chunk_documents(documents)

# Embed chunks
print("Embedding chunks with BERT...")
vectors, chunk_texts = embed_chunks_with_bert(chunks)

# Index embeddings
print("Indexing embeddings into FAISS...")
vector_store = index_embeddings(chunk_texts)

# Load LLaMA model
print("Loading LLaMA model...")
llama_llm = load_llama_model()

# Create RAG system
print("Creating RAG system...")
rag_system = create_rag_system(vector_store, llama_llm)


Loading PDF files...
Splitting documents into chunks...
Embedding chunks with BERT...


C:\Users\ayush\AppData\Local\Temp\ipykernel_18332\1484678092.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Indexing embeddings into FAISS...
Loading LLaMA model...
Creating RAG system...


In [6]:
# Run the RAG model
def run_query(query):
    print("Running query...")
    response = rag_system.run(query)
    print("\nResponse:",response)


In [9]:
user_query = "What functions are deprecated in this release of Python?" 
run_query(user_query)

Running query...

Response: Based on the context provided, the following functions are deprecated in Python 3.13:

* `soft deprecated` - This keyword is used to mark a function or method as deprecated, but it will still work and produce warnings. There are no specific functions marked as `soft deprecated` in this release of Python.
* `special method` - This keyword is used to mark a method as special, meaning it has different behavior than a regular method. There are no specific functions marked as `special method` in this release of Python.
* `static type checker` - The static type checker is a feature that checks the types of variables at compile-time. It is not specific to any function or method, but rather a feature of the language.
* `universal newlines` - This keyword is used to indicate that a string can contain any combination of newline characters (`\n`, `\r`, `\r\n`). There are no specific functions marked as `universal newlines` in this release of Python.
* `virtual environm

The model automatically answers questions based on Python, which is what we desired. Let's pass the same prompt to a vanilla llama2 model

In [ ]:
print(llama_llm.invoke("What functions are deprecated in 3.13 release of Python?"))

The model outputs a generalized answer, as it doesn't have context of python